In [ ]:
import pandas as pd
import numpy as np
import pickle
import string
import argparse
import os
import json
import random
import torch
import urllib.request
import pytreebank
from nltk.tokenize import sent_tokenize
from glob import glob
from shutil import rmtree
from pathlib import Path
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, logging
from datasets import Dataset, load_metric, concatenate_datasets

In [ ]:
def compute_metrics(eval_preds):
    metric = load_metric('accuracy')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
def load_counterfactuals(path, min_moverscore=None, max_moverscore=None, both_sentiment=False, keep_top=None, upsample=False, drop_duplicates=True):
    with open(path, 'rb') as f:
        counterfactuals = pickle.load(f)
    if min_moverscore is not None:
        counterfactuals = counterfactuals[(counterfactuals['moverscore'] >= min_moverscore)]
    if max_moverscore is not None:
        counterfactuals = counterfactuals[(counterfactuals['moverscore'] <= max_moverscore)]
    if not both_sentiment:
        counterfactuals = counterfactuals[(counterfactuals['original_label'] != counterfactuals['target_label'])]
    if drop_duplicates:
        counterfactuals = counterfactuals.drop_duplicates('counterfactual')
    if keep_top is not None:
        counterfactuals['original'] = counterfactuals['original'].apply(lambda x: ' '.join(x.split()))
        counterfactuals = counterfactuals.sort_values(keep_top, ascending=False).drop_duplicates('original')
    counterfactuals['label'] = counterfactuals['target_label'].apply(lambda x: 1 if x == 'POSITIVE' else -1)
    if upsample:
        n_class = counterfactuals['label'].value_counts().max()
        counterfactuals = pd.concat([counterfactuals[counterfactuals['label'] == -1].sample(n_class, random_state=1, replace=True), counterfactuals[counterfactuals['label'] == 1].sample(n_class, random_state=1, replace=True)])
    
    return counterfactuals

## Set random seed

In [ ]:
seed = 0

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

## Load tokenizer and model

In [ ]:
model_name = 'roberta-base'
dropout = 0.1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, classifier_dropout=dropout)

## Download datasets

In [ ]:
urllib.request.urlretrieve("https://github.com/tapilab/aaai-2021-counterfactuals/raw/main/data/ds_imdb_para.pkl", "data/ds_imdb_para.pkl")
urllib.request.urlretrieve("https://github.com/tapilab/aaai-2021-counterfactuals/raw/main/data/ds_imdb_sent.pkl", "data/ds_imdb_sent.pkl")

In [ ]:
urllib.request.urlretrieve("https://github.com/allenai/contrast-sets/raw/main/IMDb/data/test_contrast.tsv", "data/test_contrast.tsv")

## Load datasets

In [ ]:
class Counterfactual:
    def __init__(self, df_train, df_test, moniker):
        display(df_train.head(1))
        self.moniker = moniker
        self.train = df_train
        self.test = df_test

In [ ]:
train_data = {}
with open('data/ds_imdb_sent.pkl', 'rb') as f:
    imdb_sents = pickle.load(f)

train_data['original'] = {'text' : imdb_sents.train.text.values, 'label' : imdb_sents.train.label.values}

In [ ]:
test_data = {}
with open('data/ds_imdb_para.pkl', 'rb') as f:
    imdb_para = pickle.load(f)
    
test_data['IMDB Original'] = {'text' : imdb_para.test.text.values, 'label' : imdb_para.test.label.values}
test_data['IMDB Counterfactual'] = {'text' : imdb_para.test.ct_text_amt.values, 'label' : imdb_para.test.ct_label.values}

contrast = pd.read_csv('data/test_contrast.tsv', sep='\t')
test_data['IMDB Contrast Sets'] = {'text' : contrast['Text'].to_numpy(), 'label' : contrast['Sentiment'].apply(lambda x: 1 if x == 'Positive' else -1).to_numpy()}

In [ ]:
contrast = pd.read_csv('data/test_contrast.tsv', sep='\t')
text = []; label = []; index = []
for i in range(contrast.shape[0]):
    sents = sent_tokenize(contrast.iloc[i]['Text'].replace('<br />',' '))
    text += sents
    label += [1 if contrast.iloc[i]['Sentiment'] == 'Positive' else -1 for _ in sents]
    index += [i for _ in sents]
    
test_data['IMDB_Contrast_Sets'] = {'text' : np.asarray(text), 'label' : np.asarray(label), 'example_index' : np.asarray(index)}

#### Specify the type of NeuroCFs you want to evaluate

In [ ]:
NeuroCFs_type = 'NeuroCFs-np'
# NeuroCFs_type = 'NeuroCFs-1g'

In [ ]:
ctf_path = os.path.join('output', NeuroCFs_type, 'counterfactuals.pkl')
with open(ctf_path, 'rb') as f:
    counterfactuals = pickle.load(f)
    
counterfactuals = counterfactuals[counterfactuals['counterfactual'].str.split().str.len() >= 3]
train_data[NeuroCFs_type] = {'text' : np.concatenate([train_data['original']['text'], counterfactuals['counterfactual'].to_numpy()]), 
                             'label': np.concatenate([train_data['original']['label'], counterfactuals['label'].to_numpy()])}

In [ ]:
train_dataset = train_data[NeuroCFs_type]
for k in train_dataset.keys():
    train_dataset[k] = np.asarray(train_dataset[k])
train_dataset['label'][train_dataset['label'] == -1] = 0

train_dataset = Dataset.from_dict(train_dataset)
train_dataset = train_dataset.train_test_split(test_size=0.2, seed=seed)

val_dataset = train_dataset['test']
train_dataset = train_dataset['train']

train_dataset_tokenized = train_dataset.map(preprocess_function, batched=True)
val_dataset_tokenized = val_dataset.map(preprocess_function, batched=True)

## Set hyperparameters

In [ ]:
training_args = TrainingArguments(
    output_dir = 'checkpoints/',
    learning_rate = 1e-6,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 64,
    weight_decay = 0.01,
    lr_scheduler_type = 'linear',
    warmup_ratio = 0,
    evaluation_strategy = 'steps',
    save_strategy = 'steps',
    logging_strategy = 'steps',
    save_total_limit = 1,
    max_steps = 10000,
    seed = seed,
    logging_steps = 500,
    save_steps = 500,
    eval_steps = 500,
    # num_train_epochs = args.num_train_epochs,
    optim = 'adamw_hf',
    load_best_model_at_end = True,
    metric_for_best_model = 'eval_loss',
    fp16= True
)

## Train classifier

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_tokenized,
    eval_dataset=val_dataset_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 5)]
)

logging.set_verbosity_error()
train_output = trainer.train()

In [ ]:
train_output

## Evaluate model on test datasets

In [ ]:
test_results = []
for test_name in test_data.keys():
    test_dataset = test_data[test_name]
    test_dataset['label'][test_dataset['label'] == -1] = 0
    test_dataset = Dataset.from_dict(test_dataset)
    test_dataset = test_dataset.map(preprocess_function, batched=True)
    predictions = trainer.predict(test_dataset)
    pred_metrics = pd.DataFrame(predictions[2], index=[test_name])
    test_results.append(pred_metrics)

test_results = pd.concat(test_results)

In [ ]:
test_results